In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# تنظیم دستگاه برای استفاده از GPU اگر موجود باشد
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# تعریف یک شبکه عصبی کانولوشنی
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# بارگذاری و تبدیل داده‌های MNIST
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)

model = ConvNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# تابع آموزش
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# تابع تست
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

# اجرای آموزش و تست
for epoch in range(1, 10):
    train(epoch)
    test()


100%|██████████| 9912422/9912422 [00:00<00:00, 105415356.64it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 60477131.22it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32373979.62it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5466435.80it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.367244
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.266212
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.262338
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.276398
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.250542
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.163671
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.126301
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.153157
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.957612
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.760477
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.769226
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.580146
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.425440
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.479970
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.332076
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.164161
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.243704
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.185156
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.133402
Tr

/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1882, Accuracy: 9435/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.357651
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.339454
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.396041
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.623047
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.451567
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.546012
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.392053
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.428646
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.548228
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.429849
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.429736
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.513748
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.436941
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.336013
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.556510
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.561077
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.415653
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.742956
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.539366
Train Epoch: 2 [12